In [125]:
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from PIL import Image
from io import BytesIO
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

In [126]:
df = pd.read_csv('test.csv', engine ='python') # csv파일 올려서 사용
df

,학수번호,과목명,수업년도,수업학기,인문사회,자연과학,공학,예체능,국문개요,영문개요,키워드,추천강의
0,ACC2051,중급회계1,2020.0,1.0,267.0,0.0,3.0,5.0,ACC 251 재무회계 기업 재무회계의 이론적 구조를 이해하고 일반적으로 인정된 회...,Acc 251 Financial Accounting This course is in...,"회계, 재무회계, 회계정보, 의사, 신용의사","회계정보의활용, 기업윤리와사회적책임, 회계정보의이해, 스크린영어, 보이스앙상블"
1,AIX0002,AI+X:인공지능,2020.0,2.0,4.0,1.0,10.0,0.0,본 교과목은 4차산업혁명의 핵심기술인 인공지능의 개념과 원리를 이해하고 적용할 수 ...,artificial intelligence,"학문단위, 인공지능, 지능, 교과목, 학문","AI+X:머신러닝, 영화속심리학, 사이버토익2, 특수교육학개론, AI+X:R-Py컴퓨팅"
2,AIX0003,AI+X:딥러닝,2020.0,2.0,2.0,1.0,24.0,1.0,"본 교과목은 딥러닝을 개념을 이해하고, 딥러닝을 실험해 볼 수 있는 개발환경 조성과...",Introduction to Deep Learning,"딥러닝, 인공지능, 텔레콤, 동영상, 발표자료",NaN
3,AIX0004,AI+X:R-Py컴퓨팅,2020.0,2.0,5.0,6.0,17.0,0.0,본 교과목은 데이터 사이언스 및 AI 과목에 기초가 되는 R 프로그래밍 및 Pyth...,R-Py computing,"데이터, 프로그래밍, 기초, 머신러닝, 반복문과",NaN
4,AIX0005,AI+X:머신러닝,2020.0,2.0,0.0,1.0,28.0,0.0,본 교과목은 머신러닝의 이해를 돕기 위한 사례 예제 중심으로 학습한다. 머신러닝의 ...,Machine Learning,"머신러닝, 이해, 러닝, 머신, 딥러닝",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
303,SYB9112,"화폐와금융의과거,현재,그리고미래",2020.0,2.0,53.0,5.0,34.0,4.0,NaN,NaN,NaN,NaN
304,SYB9113,외국인학생을위한실용한국어,2020.0,2.0,18.0,1.0,15.0,10.0,NaN,NaN,NaN,NaN
305,SYB9115,긍정컴퓨팅을통한인간과인공지능의공존,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,SYB9125,사회복지실천과현장,2020.0,2.0,3.0,0.0,2.0,2.0,NaN,NaN,NaN,NaN


In [127]:
# df['영문개요'] = df['영문개요'].fillna('')
idx_nm_1 = df[df['영문개요'].isnull() == True].index
df = df.drop(idx_nm_1)
df

,학수번호,과목명,수업년도,수업학기,인문사회,자연과학,공학,예체능,국문개요,영문개요,키워드,추천강의
0,ACC2051,중급회계1,2020.0,1.0,267.0,0.0,3.0,5.0,ACC 251 재무회계 기업 재무회계의 이론적 구조를 이해하고 일반적으로 인정된 회...,Acc 251 Financial Accounting This course is in...,"회계, 재무회계, 회계정보, 의사, 신용의사","회계정보의활용, 기업윤리와사회적책임, 회계정보의이해, 스크린영어, 보이스앙상블"
1,AIX0002,AI+X:인공지능,2020.0,2.0,4.0,1.0,10.0,0.0,본 교과목은 4차산업혁명의 핵심기술인 인공지능의 개념과 원리를 이해하고 적용할 수 ...,artificial intelligence,"학문단위, 인공지능, 지능, 교과목, 학문","AI+X:머신러닝, 영화속심리학, 사이버토익2, 특수교육학개론, AI+X:R-Py컴퓨팅"
2,AIX0003,AI+X:딥러닝,2020.0,2.0,2.0,1.0,24.0,1.0,"본 교과목은 딥러닝을 개념을 이해하고, 딥러닝을 실험해 볼 수 있는 개발환경 조성과...",Introduction to Deep Learning,"딥러닝, 인공지능, 텔레콤, 동영상, 발표자료",NaN
3,AIX0004,AI+X:R-Py컴퓨팅,2020.0,2.0,5.0,6.0,17.0,0.0,본 교과목은 데이터 사이언스 및 AI 과목에 기초가 되는 R 프로그래밍 및 Pyth...,R-Py computing,"데이터, 프로그래밍, 기초, 머신러닝, 반복문과",NaN
4,AIX0005,AI+X:머신러닝,2020.0,2.0,0.0,1.0,28.0,0.0,본 교과목은 머신러닝의 이해를 돕기 위한 사례 예제 중심으로 학습한다. 머신러닝의 ...,Machine Learning,"머신러닝, 이해, 러닝, 머신, 딥러닝",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
286,LIT0013,학기제현장실습(전공핵심),2020.0,2.0,28.0,3.0,43.0,7.0,"대학-산업체간 인력양성, skill mismatch 문제를 해결하는 방안 중 하나가...",ERICA Work Placement is an educational curricu...,NaN,NaN
287,LIT0014,학기제현장실습(전공심화),2020.0,2.0,29.0,2.0,50.0,7.0,"대학-산업체간 인력양성, skill mismatch 문제를 해결하는 방안 중 하나가...",ERICA Work Placement is an educational curricu...,NaN,NaN
290,SYB9031,문화기술과사회변동,2020.0,2.0,58.0,2.0,23.0,10.0,문화와 기술의 결합을 의미하는 CT(Culture Technology)는 문화콘텐츠...,"· CT (Culture Technology), which means a combi...",NaN,NaN
296,SYB9082,색채심리와현대생활,2020.0,2.0,17.0,7.0,20.0,5.0,"색에 대한 기초이론을 습득하고, 생활 속에서 색채를 이용하여 사람의 다양한 감정을 ...",Students will learn the basic theory of color ...,NaN,NaN


In [128]:
import nltk
from nltk import word_tokenize
nltk.download('stopwords')

def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

df['cleaned'] = df['영문개요'].apply(_removeNonAscii)
df['cleaned'] = df.cleaned.apply(make_lower_case)
df['cleaned'] = df.cleaned.apply(remove_stop_words)
df['cleaned'] = df.cleaned.apply(remove_punctuation)
df['cleaned'] = df.cleaned.apply(remove_html)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [129]:
df['cleaned'][:5]

0    acc financial accounting course intermediate a...
1                              artificial intelligence
2                           introduction deep learning
3                                       r py computing
4                                     machine learning
Name: cleaned, dtype: object

In [130]:
# df['cleaned'].replace('', np.nan, inplace=True)
# df = df[df['cleaned'].notna()]
print('전체 문서의 수 :',len(df))

전체 문서의 수 : 214


In [131]:
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

In [8]:
urllib.request.urlretrieve("https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz", \
                           filename="GoogleNews-vectors-negative300.bin.gz")
word2vec_model = Word2Vec(size = 300, window=5, min_count = 2, workers = -1)
word2vec_model.build_vocab(corpus)
word2vec_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz', lockf=1.0, binary=True)
word2vec_model.train(corpus, total_examples = word2vec_model.corpus_count, epochs = 15)

(0, 0)

In [132]:
def vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in word2vec_model.wv.vocab:
                count += 1
                # 해당 문서에 있는 모든 단어들의 벡터값을 더한다.
                if doc2vec is None:
                    doc2vec = word2vec_model[word]
                else:
                    doc2vec = doc2vec + word2vec_model[word]

        if doc2vec is not None:
            # 단어 벡터를 모두 더한 벡터의 값을 문서 길이로 나눠준다.
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [133]:
len(df['cleaned'])
df.to_csv('result.csv', index=False)

In [134]:
document_embedding_list = vectors(df['cleaned'])
print('문서 벡터의 수 :',len(document_embedding_list))

문서 벡터의 수 : 213


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [135]:
from sklearn.metrics.pairwise import linear_kernel
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
# print('코사인 유사도 매트릭스의 크기 :',cosine_similarities.shape)

# cosine_sim = linear_kernel(document_embedding_list, document_embedding_list)


In [136]:
# def recommendations(title, cosine_sim=cosine_sim):
#     # 책의 제목을 입력하면 해당 제목의 인덱스를 리턴받아 idx에 저장.
#     indices = pd.Series(df.index, index = df['과목명']).drop_duplicates()    
#     idx = indices[title]

#     # 입력된 책과 줄거리(document embedding)가 유사한 책 5개 선정.
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
#     sim_scores = sim_scores[1:6]

#     # 가장 유사한 책 5권의 인덱스
#     book_indices = [i[0] for i in sim_scores]

#     return df['과목명'].iloc[book_indices]

def recommendations(title):
    # 책의 제목을 입력하면 해당 제목의 인덱스를 리턴받아 idx에 저장.
    indices = pd.Series(df.index, index = df['과목명']).drop_duplicates()    
    idx = indices[title]

    # 입력된 책과 줄거리(document embedding)가 유사한 책 5개 선정.
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:6]

    # 가장 유사한 책 5권의 인덱스
    book_indices = [i[0] for i in sim_scores]

    return df['과목명'].iloc[book_indices]

In [137]:
# # df['과목명'][214]
# df['newidx'] = 0 # 0으로만 이루어진 새로운 칼럼 생성

# for i in range(len(df)): 
#   df['newidx'][i] = i # newidx를 0~214의 숫자로 차례로 채움

# # df = df.set_index("newidx") # newidx를 df의 새로운 인덱스로 만듦
# df['newidx']

In [138]:
# dummy_data = {
#     '학수번호': [0],

# }
# new_df = pd.DataFrame(dummy_data)
# df = pd.concat([df, new_df])

df['newidx'] = range(0,len(df))
df = df.set_index("newidx") # newidx를 df의 새로운 인덱스로 만듦
df

,학수번호,과목명,수업년도,수업학기,인문사회,자연과학,공학,예체능,국문개요,영문개요,키워드,추천강의,cleaned
newidx,,,,,,,,,,,,,
0,ACC2051,중급회계1,2020.0,1.0,267.0,0.0,3.0,5.0,ACC 251 재무회계 기업 재무회계의 이론적 구조를 이해하고 일반적으로 인정된 회...,Acc 251 Financial Accounting This course is in...,"회계, 재무회계, 회계정보, 의사, 신용의사","회계정보의활용, 기업윤리와사회적책임, 회계정보의이해, 스크린영어, 보이스앙상블",acc financial accounting course intermediate a...
1,AIX0002,AI+X:인공지능,2020.0,2.0,4.0,1.0,10.0,0.0,본 교과목은 4차산업혁명의 핵심기술인 인공지능의 개념과 원리를 이해하고 적용할 수 ...,artificial intelligence,"학문단위, 인공지능, 지능, 교과목, 학문","AI+X:머신러닝, 영화속심리학, 사이버토익2, 특수교육학개론, AI+X:R-Py컴퓨팅",artificial intelligence
2,AIX0003,AI+X:딥러닝,2020.0,2.0,2.0,1.0,24.0,1.0,"본 교과목은 딥러닝을 개념을 이해하고, 딥러닝을 실험해 볼 수 있는 개발환경 조성과...",Introduction to Deep Learning,"딥러닝, 인공지능, 텔레콤, 동영상, 발표자료",NaN,introduction deep learning
3,AIX0004,AI+X:R-Py컴퓨팅,2020.0,2.0,5.0,6.0,17.0,0.0,본 교과목은 데이터 사이언스 및 AI 과목에 기초가 되는 R 프로그래밍 및 Pyth...,R-Py computing,"데이터, 프로그래밍, 기초, 머신러닝, 반복문과",NaN,r py computing
4,AIX0005,AI+X:머신러닝,2020.0,2.0,0.0,1.0,28.0,0.0,본 교과목은 머신러닝의 이해를 돕기 위한 사례 예제 중심으로 학습한다. 머신러닝의 ...,Machine Learning,"머신러닝, 이해, 러닝, 머신, 딥러닝",NaN,machine learning
...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,LIT0013,학기제현장실습(전공핵심),2020.0,2.0,28.0,3.0,43.0,7.0,"대학-산업체간 인력양성, skill mismatch 문제를 해결하는 방안 중 하나가...",ERICA Work Placement is an educational curricu...,NaN,NaN,erica work placement educational curriculum de...
210,LIT0014,학기제현장실습(전공심화),2020.0,2.0,29.0,2.0,50.0,7.0,"대학-산업체간 인력양성, skill mismatch 문제를 해결하는 방안 중 하나가...",ERICA Work Placement is an educational curricu...,NaN,NaN,erica work placement educational curriculum de...
211,SYB9031,문화기술과사회변동,2020.0,2.0,58.0,2.0,23.0,10.0,문화와 기술의 결합을 의미하는 CT(Culture Technology)는 문화콘텐츠...,"· CT (Culture Technology), which means a combi...",NaN,NaN,ct culture technology means combination cultur...


In [139]:
recommendations('실용중국어')

newidx
186       프레젠테이션전략
200    아카데믹한국어읽기쓰기
90          초급스페인어
201       한국어듣기말하기
205        일본사회와문화
Name: 과목명, dtype: object

In [142]:
indices = pd.Series(df.index, index=df['과목명']).drop_duplicates()
# print(indices.head())

for i in range(len(df)-1):
  print(i+1,'번째 과목명: ', indices.index[i])
  print(recommendations(indices.index[i]))
  print("==================================")

1 번째 과목명:  중급회계1
newidx
42              스크린영어
6             회계정보의이해
11               창의역량
87           계절제현장실습2
112    나를좋아하게하는커뮤니케이션
Name: 과목명, dtype: object
2 번째 과목명:  AI+X:인공지능
newidx
153    디지털미디어리터러시
176    융합과함께하는ICT
9          경영자료분석
167        실용스페인어
196       치료음악의세계
Name: 과목명, dtype: object
3 번째 과목명:  AI+X:딥러닝
newidx
189            교양심화프랑스어
153          디지털미디어리터러시
198            K-POP한국어
183              키치와팝아트
52     채움과비움:나이듦의인문학적이해
Name: 과목명, dtype: object
4 번째 과목명:  AI+X:R-Py컴퓨팅
newidx
27     NCS기반채용의이해와직업요구역량(클러스터)
119                     드론SW제작
175                      웹툰의세계
125              영어로만나는YouTube
23       누구나할수있는스마트폰앱만들기(클러스터)
Name: 과목명, dtype: object
5 번째 과목명:  AI+X:머신러닝
newidx
57                글로벌비즈니스와창업기회
198                   K-POP한국어
189                   교양심화프랑스어
134                   뉴스영어따라잡기
27     NCS기반채용의이해와직업요구역량(클러스터)
Name: 과목명, dtype: object
6 번째 과목명:  경영통계분석1
newidx
9         경영자료분석
194     젠더혁신프로젝트
117      독립영화의이해
87      계절제현장실습2
129    교양태권도및

IndexError: ignored

In [111]:
df['과목명'][213]

'선택형Co-op'